# Personalized Programming Tutor - Week 1 Project

## Project Objective

Build a **personalized tutor** that takes a technical question about code and responds with a detailed and educational explanation.

### What does this tutor do?

- Explains complex code in a simple and structured way
- Provides examples and analogies
- Teaches the "why" behind code patterns
- Responds in well-formatted Markdown
- Supports **streaming** to see responses in real-time

### Features:

- **Flexible**: Switch between Ollama (local, free) and OpenAI (cloud)
- **Interactive**: Streaming mode with typewriter effect
- **Educational**: Designed specifically for learning

---

**Instructions**: Execute the cells in order and modify the `question` variable to ask new questions.

In [1]:
# Imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# CONFIGURATION: Change here to test different providers
# Options: 'ollama' or 'openai'
USE_PROVIDER = 'ollama'  # Change to 'openai' to use OpenAI

print(f"Selected provider: {USE_PROVIDER.upper()}")

Selected provider: OLLAMA


In [25]:
# Initialize and constants

# Load configuration from global .env
load_dotenv(dotenv_path='/workspace/.env', override=True)

# Use provider selected in previous cell (or from .env as fallback)
llm_provider = USE_PROVIDER if 'USE_PROVIDER' in globals() else os.getenv('LLM_PROVIDER', 'ollama')

if llm_provider == 'ollama':
    # OLLAMA CONFIGURATION (Local)
    ollama_base_url = os.getenv('OLLAMA_BASE_URL')
    ollama_api_key = os.getenv('OLLAMA_API_KEY')
    #ollama_model = os.getenv('OLLAMA_MODEL')
    ollama_model = 'qwen3-coder:480b-cloud'
    
    print("=" * 50)
    print("Using OLLAMA (Local)")
    print("=" * 50)
    print(f"   Base URL: {ollama_base_url}")
    print(f"   Model: {ollama_model}")
    print(f"   Cost: FREE")
    print("=" * 50)
    
    # Create OpenAI client pointing to Ollama
    openai = OpenAI(
        base_url=f"{ollama_base_url}/v1",
        api_key=ollama_api_key
    )
    MODEL = ollama_model
    
else:
    # OPENAI CONFIGURATION (Cloud)
    api_key = os.getenv('OPENAI_API_KEY')
    
    print("=" * 50)
    print("Using OPENAI (Cloud API)")
    print("=" * 50)
    
    if api_key and api_key.startswith('sk-proj-') and len(api_key) > 50:
        print(f"   API Key: sk-proj-...{api_key[-8:]}")
        print(f"   Model: gpt-4o-mini")
        print(f"   Cost: ~$0.15 / 1M tokens input")
        print("   Status: Configured correctly")
    else:
        print("   Status: Invalid or missing API Key")
        print("   Check the .env file")
    
    print("=" * 50)
    
    openai = OpenAI(api_key=api_key)
    MODEL = 'gpt-4o-mini'

Using OLLAMA (Local)
   Base URL: http://192.168.80.200:11434
   Model: qwen3-coder:480b-cloud
   Cost: FREE


In [26]:
# Call LLM API with streaming support

def callModel(ask, use_stream=False):
    """
    Calls the LLM model with the provided messages
    
    Args:
        ask: List of messages with format [{"role": "...", "content": "..."}]
        use_stream: If True, displays response in real-time (typewriter effect)
    
    Returns:
        The complete response content
    """
    if use_stream:
        stream = openai.chat.completions.create(
            model=MODEL,
            messages=ask,
            stream=True  # Enable streaming
        )
        
        response = ""
        display_handle = None
        
        for chunk in stream:
            content = chunk.choices[0].delta.content or ''
            response += content
            
            if display_handle is None and response:
                display_handle = display(Markdown(response), display_id=True)
            elif display_handle is not None:
                update_display(Markdown(response), display_id=display_handle.display_id)
        
    else:
        response = openai.chat.completions.create(
            model=MODEL,
            messages=ask
        )
        result = response.choices[0].message.content
        display(Markdown(result))

In [27]:
# Define our system prompt - PERSONALIZED TUTOR

system_prompt = """
You are an expert programming tutor with deep knowledge in Python, JavaScript, and software engineering.

Your teaching style:
- Explain concepts clearly and step by step
- Use analogies and real-world examples
- Break down complex code into understandable parts
- Highlight best practices and common pitfalls
- Encourage learning by explaining the "why" behind the code

Response format:
- Always respond in well-formatted Markdown
- Use headers, lists, and code blocks appropriately
- Do NOT wrap your entire response in markdown code fences (```)
- Include examples when helpful
- Be concise but thorough

Your goal is to help students truly understand programming concepts, not just memorize syntax.
"""

In [28]:
# Get the technical question from user input

print("=" * 60)
print("PERSONALIZED PROGRAMMING TUTOR")
print("=" * 60)
print("\nEnter the code you want to understand.")
print("\nExample:")
print('  yield from {book.get("author") for book in books if book.get("author")}')
print("\nYour question (paste your code below):")
print("-" * 60)

question = input()

PERSONALIZED PROGRAMMING TUTOR

Enter the code you want to understand.

Example:
  yield from {book.get("author") for book in books if book.get("author")}

Your question (paste your code below):
------------------------------------------------------------


 yield from {book.get("author") for book in books if book.get("author")}


In [29]:
# Define user prompt using the question variable

user_prompt = f"""
Please explain the following code in detail:

```python
{question}
```

Include:
1. What this code does
2. How it works step by step
3. Why someone would use this approach
4. Any important concepts or patterns involved
"""

# Build messages for the LLM
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [30]:
# Execute the tutor with the question

# Option 1: Without streaming (shows complete result at the end)
# callModel(messages)

# Option 2: With streaming (typewriter effect - recommended)
callModel(messages, use_stream=True)

# Understanding `yield from` with Set Comprehension

## What This Code Does

This code **extracts unique author names** from a collection of books and yields them one by one. It's a generator expression that:
- Filters books to only those with author information
- Extracts the author names
- Ensures uniqueness (no duplicate authors)
- Yields each unique author individually

## How It Works Step by Step

Let me break this down into its components:

### 1. The Set Comprehension `{...}`
```python
{book.get("author") for book in books if book.get("author")}
```

This creates a **set** (unique collection) using this logic:
- **Iteration**: `for book in books` - goes through each book
- **Filter**: `if book.get("author")` - only includes books that have an author
- **Extraction**: `book.get("author")` - gets the author name from each book

### 2. The `yield from` Statement
```python
yield from {set_of_authors}
```

Instead of yielding the entire set at once, `yield from` **delegates** to the set and yields each element individually.

## Complete Example

```python
def get_unique_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

# Example usage
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3", "author": "Author A"},  # Duplicate
    {"title": "Book 4"},  # No author
    {"title": "Book 5", "author": "Author C"}
]

authors = list(get_unique_authors(books))
print(authors)  # ['Author A', 'Author B', 'Author C'] (order may vary - it's a set!)
```

## Why Use This Approach?

### Advantages:
1. **Memory Efficient**: Generates authors one at a time instead of creating a full list
2. **Automatic Deduplication**: Sets inherently remove duplicates
3. **Clean and Concise**: Combines filtering, extraction, and deduplication in one line
4. **Lazy Evaluation**: Authors are only processed when requested

### Alternative Approaches:
```python
# More verbose but clearer for beginners
def get_unique_authors_verbose(books):
    seen_authors = set()
    for book in books:
        author = book.get("author")
        if author and author not in seen_authors:
            seen_authors.add(author)
            yield author

# Creates full list in memory (less efficient)
def get_unique_authors_list(books):
    return list({book.get("author") for book in books if book.get("author")})
```

## Important Concepts and Patterns

### 1. **Set Comprehension**
```python
{expression for item in iterable if condition}
```
- Creates a set automatically removing duplicates
- More efficient than list comprehension + set conversion

### 2. **`yield from` Pattern**
- **Delegation**: Delegates to another iterable
- **Flattening**: Useful for yielding elements from nested structures
- **Memory Efficiency**: Maintains generator benefits

### 3. **Defensive Programming with `.get()`**
```python
book.get("author")  # Returns None if key doesn't exist
book["author"]      # Raises KeyError if key doesn't exist
```

### 4. **Truthiness Filtering**
```python
if book.get("author")  # Filters out None, empty strings, etc.
```

## Common Pitfalls

⚠️ **Order is not guaranteed** - sets are unordered, so author yield order may vary
⚠️ **Empty values** - If author field exists but is empty string (`""`), it gets filtered out
⚠️ **Type safety** - assumes all author values are hashable (strings, not lists/dicts)

## Real-World Analogy

Think of this like a **library card catalog system**:
1. You have a pile of books (`books`)
2. You only want cards for books that have author information (`if book.get("author")`)
3. You extract the author names and put each unique name on a separate index card (`set comprehension`)
4. You then feed these cards one by one into a sorting machine (`yield from`)

This approach is elegant for scenarios where you need unique values processed individually while maintaining memory efficiency!

------------------------------------------------------------------------------------------------------------------------------

## How to Use This Personalized Tutor

To ask a question:

1. **Execute cells 1-6** to set up the environment
2. **Execute cell 7** and enter your code when prompted
3. **Execute cells 8 and 9** to get the explanation

### Examples of questions you can ask:

```python
# Example 1: Generators
yield from {book.get("author") for book in books if book.get("author")}

# Example 2: List comprehension
[x**2 for x in range(10) if x % 2 == 0]

# Example 3: Decorators
@decorator
def function(): pass

# Example 4: Context managers
with open("file.txt") as f:
    data = f.read()

# Example 5: Lambda and map
list(map(lambda x: x * 2, [1, 2, 3]))
```

**Tip**: Use `use_stream=True` in cell 9 to see the response in real-time!